EasyOCR

In [4]:
import easyocr

reader = easyocr.Reader(['es']) 

path_img = reader.readtext('matricula.jpg')

for (bbox, text, prob) in path_img:
    # Coordenadas en orden 
    (top_left, top_right, bottom_right, bottom_left) = bbox
    print(f'\nTexto: {text}\nProbabilidad: {prob:.2f}\nContenedor: {tuple(map(int, top_left)),tuple(map(int, bottom_right))}')




Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



Texto: 072L HPH
Probabilidad: 0.40
Contenedor: ((11, 1), (126, 38))


In [ ]:
import cv2
import easyocr
from ultralytics import YOLO
import csv
import os

try:
    model_placas = YOLO(r'runs\detect\matricula_yolo11n\weights\best.pt') 
except Exception as e:
    print(f"Error cargando tu modelo de matrículas: {e}")
    exit()


try:
    model_general = YOLO('yolo11n.pt') 
except Exception as e:
    print(f"Error cargando el modelo 'yolo11n.pt': {e}")
    exit()

try:
    reader = easyocr.Reader(['en'], gpu=True) 
except Exception as e:
    print(f"Error cargando easyOCR: {e}")
    exit()
    
ALLOWED_CHARACTERS = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'


CLASES_GENERALES_INTERES = [0, 2] 
ID_PERSONA = 0 
ID_COCHE = 2    


video_path = r'C:\Users\Alfredo\Downloads\C0142.MP4' 
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: No se pudo abrir el video en {video_path}")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_path = 'video_salida_tracking.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

csv_path = 'resultados_tracking.csv'
csv_header = [
    'fotograma', 
    'track_id', 
    'tipo_objeto', 
    'confianza', 
    'x1', 'y1', 'x2', 'y2', 
    'texto_matricula'
]

try:
    csv_file = open(csv_path, 'w', newline='', encoding='utf-8')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(csv_header)
except IOError as e:
    print(f"Error abriendo el archivo CSV: {e}")
    exit()

print(f"Guardando video en: {output_path}")
print(f"Guardando datos CSV en: {csv_path}")

frame_counter = 0 

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Fin del video.")
        break
        
    frame_counter += 1

    try:
        results_general = model_general.track(
            frame, 
            classes=CLASES_GENERALES_INTERES, 
            persist=True, 
            verbose=False
        )[0]
    except Exception as e:
        print(f"Error en tracking general: {e}")
        continue

    try:
        results_placas = model_placas.track(
            frame, 
            persist=True, 
            verbose=False
        )[0]
    except Exception as e:
        print(f"Error en tracking de matrículas: {e}")
        continue

    if results_general.boxes.id is not None:
        boxes = results_general.boxes
        for i in range(len(boxes)):
            box = boxes[i]
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            track_id = int(box.id[0]) 

            if cls == ID_PERSONA:
                nombre_clase = 'persona'
                color = (255, 0, 0) 
            elif cls == ID_COCHE:
                nombre_clase = 'coche'
                color = (0, 0, 255) 
            else:
                continue
            

            label = f"ID: {track_id} {nombre_clase}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)


            csv_row = [frame_counter, track_id, nombre_clase, f"{conf:.2f}", x1, y1, x2, y2, '']
            csv_writer.writerow(csv_row)
            
    if results_placas.boxes.id is not None:
        boxes_placas = results_placas.boxes
        for i in range(len(boxes_placas)):
            box = boxes_placas[i]
            x1_placa, y1_placa, x2_placa, y2_placa = map(int, box.xyxy[0])
            conf_placa = float(box.conf[0])
            track_id_placa = int(box.id[0])
            nombre_clase_placa = 'matricula' 
            
            texto_matricula = "" 
            try:
                matricula_img = frame[y1_placa:y2_placa, x1_placa:x2_placa]
                if matricula_img.size > 0:
                    matricula_gray = cv2.cvtColor(matricula_img, cv2.COLOR_BGR2GRAY)
                    ocr_result = reader.readtext(
                        matricula_gray, 
                        detail=0,
                        allowlist=ALLOWED_CHARACTERS,
                        paragraph=False 
                    )
                    if ocr_result:
                        texto_matricula = "".join(ocr_result).upper()
            except Exception as e:
                print(f"Error en OCR: {e}") 


            color_placa = (0, 255, 0) 
            label_placa = f"ID: {track_id_placa} [{texto_matricula}]"
            cv2.rectangle(frame, (x1_placa, y1_placa), (x2_placa, y2_placa), color_placa, 2)
            cv2.putText(frame, label_placa, (x1_placa, y1_placa - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color_placa, 2)


            csv_row_placa = [frame_counter, track_id_placa, nombre_clase_placa, f"{conf_placa:.2f}", x1_placa, y1_placa, x2_placa, y2_placa, texto_matricula]
            csv_writer.writerow(csv_row_placa)



    out.write(frame)
    cv2.imshow('Deteccion con Tracking (YOLO + easyOCR)', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


print("Limpiando y guardando archivos...")
cap.release()
out.release() 
csv_file.close() 
cv2.destroyAllWindows()
print("¡Proceso completado!")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Guardando video en: video_salida_tracking.mp4
Guardando datos CSV en: resultados_tracking.csv
Fin del video.
Limpiando y guardando archivos...
¡Proceso completado!
